# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.126594e+02     1.811673e+00
 * time: 0.2995798587799072
     1     1.045863e+01     9.311585e-01
 * time: 1.8902819156646729
     2    -1.173817e+01     1.082801e+00
 * time: 2.432687997817993
     3    -3.421427e+01     7.858121e-01
 * time: 3.242224931716919
     4    -4.760661e+01     5.069613e-01
 * time: 4.0558178424835205
     5    -5.689335e+01     2.151808e-01
 * time: 4.8524558544158936
     6    -5.980048e+01     1.368355e-01
 * time: 5.403043985366821
     7    -6.091857e+01     4.905945e-02
 * time: 5.941468954086304
     8    -6.133020e+01     7.647513e-02
 * time: 6.479715824127197
     9    -6.162721e+01     3.804444e-02
 * time: 7.024501800537109
    10    -6.183860e+01     3.226668e-02
 * time: 7.562685012817383
    11    -6.199964e+01     1.976373e-02
 * time: 8.100736856460571
    12    -6.206091e+01     1.860810e-02
 * time: 8.644771814346313
    13    -6.212270e+01     1.500795e-02
 * time: 9.183069944381714
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557703
    AtomicNonlocal      14.8522660
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485380 
    Xc                  -19.3336823

    total               -62.261666459744
